In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Define node features (2 nodes, 3-dimensional feature vectors)
x = torch.tensor([[1.0, 2.0, 3.0], 
                  [4.0, 5.0, 6.0]], dtype=torch.float)

# Define edges (edge_index), including multiple edges between nodes 0 and 1
edge_index = torch.tensor([
    [0, 1, 0, 1],  # Source nodes
    [1, 0, 1, 0]   # Target nodes
], dtype=torch.long)

# Create graph data
data = Data(x=x, edge_index=edge_index)


edge_attr = torch.tensor([1.0, 1.0, 2.0, 2.0], dtype=torch.float)





tensor([[[0., 2.],
         [2., 0.]]])